In [1]:
import tensorflow as tf
import pickle
import numpy as np
import time
import load_cifar

In [2]:
tf.reset_default_graph()
val_data, val_labels = load_cifar.load_preprocessed_validation_batch()
test_data, test_labels = load_cifar.load_pre_test_batch(10000)

<h1>Hyperparameter</h1>

In [3]:
num_epochs = 10
batch_size = 32
lr = 0.001
dropout_rate = 0.3

<h1>Placeholder</h1>

In [4]:
x_data = tf.placeholder(tf.float32, shape=(None, 32, 32, 3))
y_labels = tf.placeholder(tf.float32, shape=(None, 10))
training = tf.placeholder_with_default(False, shape=(),name='training')
#regularizer = tf.contrib.layers.l2_regularizer(0.001)

<h1>LeNet-5</h1>

In [5]:
# c1: convolution2d
conv1_b = tf.Variable(tf.zeros(6))
conv1 = tf.layers.conv2d(x_data, filters=6, kernel_size=5, activation = tf.nn.relu,padding = 'VALID')
#conv1_bn_act = tf.nn.relu(conv1_bn)
conv1_out = conv1 + conv1_b
pool1 = tf.layers.max_pooling2d(conv1_out, pool_size=2, strides=2,padding = 'VALID')
pool1_bn = tf.layers.batch_normalization(pool1)

# c2: convolution 2d size (5,5, 10)
conv2_b = tf.Variable(tf.zeros(16))
conv2 = tf.layers.conv2d(pool1_bn, filters=16, kernel_size=5,activation = tf.nn.relu,padding = 'VALID')
#conv2_bn_act = tf.nn.relu(conv2_bn)
conv2_out = conv2 + conv2_b
pool2 = tf.layers.average_pooling2d(conv2_out, pool_size=2, strides=2,padding = 'VALID')
pool2_bn = tf.layers.batch_normalization(pool2)

# c4: 1x1 conv layer
conv3 = tf.layers.conv2d(pool2_bn, filters=120, kernel_size=5,activation = tf.nn.relu,padding = 'VALID')
conv3_bn = tf.layers.batch_normalization(conv3)
conv3_flat = tf.layers.flatten(conv3_bn)


fc = tf.layers.dense(conv3_flat, units=84, activation = tf.nn.relu)
fc_drop = tf.layers.dropout(fc,dropout_rate,training=training)
fc_bn = tf.layers.batch_normalization(fc_drop)
#fc_bn_act = tf.nn.relu(fc_bn)
out = tf.layers.dense(fc_bn, units=10)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use keras.layers.average_pooling2d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


<h1>Cost and Optimization</h1>

In [6]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_labels, logits=out))
#l2_loss = tf.losses.get_regularization_loss()
#loss = loss + l2_loss
opt = tf.train.AdamOptimizer(learning_rate = lr)
minimize = opt.minimize(loss)

# compare prediction accuracy 
correct_pred = tf.equal(tf.argmax(tf.nn.softmax(out),1),tf.argmax(y_labels,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')

<h1>Training, validation and testing</h1>
<h2>Train your model only 10 epochs.</h2>
<h2>1.Print out validation accuracy after each training epoch</h2>
<h2>2.Print out training time for each training epoch</h2>
<h2>3.Print out testing accuracy</h2>

In [7]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(num_epochs):
        ts = time.time()
        batch_num = 1
        while batch_num < 6:
            #print('Training for Batch: \t' + str(batch_num))
            training_data = load_cifar.load_preprocessed_training_batch(batch_num, batch_size)
            for data, labels in training_data:
                f_dict = {x_data: np.reshape(data, (-1, 32, 32, 3)), y_labels: labels, training: True}
                sess.run(minimize, feed_dict=f_dict)
            batch_num += 1
        te = time.time()
        val_acc = sess.run(accuracy, feed_dict={x_data:np.reshape(val_data, (-1, 32, 32, 3)), y_labels:val_labels})
        print("Epoch " + str(i+1) + ", Validation Accuracy:{:.4f}".format(val_acc)+ ", Time: {:.3f}s\t".format(te-ts))
        
    print('test set accuracy is: \t')
    test_acc = sess.run(accuracy, feed_dict={x_data:np.reshape(test_data, (-1, 32, 32, 3)), y_labels:test_labels})
    print(test_acc)

Epoch 1, Validation Accuracy:0.4302, Time: 7.066s	
Epoch 2, Validation Accuracy:0.4642, Time: 5.840s	
Epoch 3, Validation Accuracy:0.5098, Time: 5.799s	
Epoch 4, Validation Accuracy:0.5340, Time: 5.743s	
Epoch 5, Validation Accuracy:0.5398, Time: 5.694s	
Epoch 6, Validation Accuracy:0.5484, Time: 5.680s	
Epoch 7, Validation Accuracy:0.5592, Time: 5.685s	
Epoch 8, Validation Accuracy:0.5670, Time: 5.677s	
Epoch 9, Validation Accuracy:0.5660, Time: 5.696s	
Epoch 10, Validation Accuracy:0.5716, Time: 5.685s	
test set accuracy is: 	
0.573
